In [ ]:
import warnings
from pprint import pprint

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import torch

Распределите в команде гпу, задайте свой номер.

In [ ]:
cuda_device = 0

Импортируем полезную либу `attrdict`. Чем она хороша: позволяет обращаться к элементам словаря, как к его атрибутам

In [ ]:
from attrdict import AttrDict

# Работа с аудио и текстом

В распознавании речи нейронная сеть обучается на парах аудио+текст.

Давайте научимся открывать аудиофайлы и подготавливать их для работы с нейронной сетью.



На лекциях мы обсуждали, что аудио может быть записано с разной частотой дискретизации (sample rate), но для обучения нейронной сети обычно все аудио приводят к одной частоте дискретизации. (В этом проекте мы будем использовать sample rate 8000).

Так же аудио может быть одноканальным или многоканальным. Проблема с многоканальным аудио может быть в том, что речь в двух каналах может звучать одновременно, такие аудио надо разделять на 2 дорожки. В нашем же случае данные подобраны таким образом, что такой ситуации не возникет, поэтому несколько каналов можно просто усреднить.


Будем использовать библиотеку `torchaudio` [docs](https://pytorch.org/audio/).

Реализуйте функцию `open_audio`, которая открывает аудио (искать [тут](https://pytorch.org/audio/stable/torchaudio.html)), усредняет аудио по всем каналам (это можно сделать обычным усреднением) и приводит к необходимой частоте дискретизации (искать [тут](https://pytorch.org/audio/stable/transforms.html)).

In [ ]:
import torchaudio

In [ ]:
def open_audio(audio_path, desired_sample_rate):
    
    """ Open and resample audio, average across channels
        Inputs:
            audio_path: str, path to audio
            desired_sample_rate: int, the sampling rate to which we would like to convert the audio
        Returns:
            audio: 1D tensor with shape (num_timesteps)
            audio_len: int, len of audio
    """
    
    
    ### write your code here ###
    
    return audio, audio_len


Запустите тесты, чтобы проверить себя.
<img src="images/tests_are_all_we_need.png" width="400" height="600">


**Перед каждым запуском тестов не забывайте сохранять ноутбук.**

In [ ]:
! pytest tests/test_open_audio.py

Теперь давайте откроем аудио, и послушаем, что у нас получилось.

In [ ]:
from IPython.display import Audio

In [ ]:
sample_rate=8000
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
Audio(data=audio.numpy(), rate=sample_rate)

Так же послушать аудио можно через путь к аудио файлу.

Можно заметить, что звучание немного поменялось. Это произошло из-за того, что мы поменяли оригинальный sample rate 48000Hz на 8000Hz.

In [ ]:
Audio('test_files/test_audio.mp3')

Speech2text  —  это не только speech, но и text, поэтому теперь давайте поговорим о предобработке текста. 

Первым шагом необходимо привести текст к нижнему регистру и удалить пунктуацию. Если на этом этапе в тексте содержатся символы кроме русского алфавита и пробела (например, цифры), то такие примеры лучше вообще убрать из обучающей выборки (если просто убрать символ из строки, то может нарушиться соответствие аудио-текст).

В размеченных данных для обучения и валидации уже произедена очистка и удаление ненужных символов, поэтому нам надо только извлечь токены из текста.

В качестве токенов для обучения нейронной сети будем использовать буквы русского алфавита и пробел. Так же, как было сказано в лекциях, нам потребуется специальный символ `<blank>` для построения выравниваний.

In [ ]:
alphabet = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к',
            'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц',
            'ч', 'ш', 'щ', 'ь', 'ы', 'ъ', 'э', 'ю', 'я',
            ' ', '<blank>']

Но, естественно, в таком виде передавать буквы в нейронную сеть мы не можем — их нужно закодировать в числа. Для этого будем использовать уже готовую функцию Vocab.

Обратите внимание, что Vocab добавляет дополнительный токен `<unk>`, ответственный за все символы, которых нет в словаре.

In [ ]:
from vocabulary import Vocab
vocab = Vocab(alphabet)

Попрактикуйтесь с Vocab. Для того, чтобы понять все возможности Vocab можно заглянуть в код vocabulary.py и почитать докстринги.

* переведите "привет" в индексы
* переведите [11, 0, 11, 33, 4, 5, 12, 0] в текст
* что будет если перевести в индексы слово "hi"?
* а [44, 5] в текст?


Какие еще у Vocab возможности?

Если в процессе исследования вы изменили текущий vocab (путем добавления нового токена, например), не забудьте вернуть vocab к начальному состоянию  `vocab = Vocab(alphabet)`

Давайте извлечем из Vocab и сохраним в переменные два важных значения - длину алфавита и значения индекса `<blank>`.
    
Эти значения нам еще много раз пригодятся.

In [ ]:
def get_num_tokens(vocab):
    ### write your code here ###
    return num_tokens

def get_blank_index(vocab):
    ### write your code here ###
    return blank_index

In [ ]:
! pytest tests/test_num_tokens.py
! pytest tests/test_blank_index.py

In [ ]:
num_tokens = get_num_tokens(vocab) 
blank_index = get_blank_index(vocab)

# Подготовка датасета

Обучать нейронную сеть мы будем на подготовленном датасете. Для удобства мы сгруппировали обучающие и валидационные данные в текстовые файлы, где в каждой строке через запятую указан путь к файлу, соответствующий текст, длина аудио (в секундах).


Давайте реализуем функцию AudioDataset, которая подготавливает текст и аудио для каждого элемента датастета.


На этих данных мы будем обучать модель с помощью ctc лосса, а мы уже знаем, что для обучения с ctc аудио лучше отсортировать по длине - так модели будет проще обучиться.

Изучить документацию по Dataset и DataLoader (пригодится далее) можно найти [тут](https://pytorch.org/docs/stable/data.html).

In [ ]:
import pandas as pd

In [ ]:
 class AudioDataset(torch.utils.data.Dataset):
    def __init__(self,
                 dataset_path,
                 vocab,
                 sample_rate=8000,
                 ):
        self.vocab = vocab
        self.sample_rate = sample_rate
        data = pd.read_csv(dataset_path, header=None, names=['audio_path', 'text', 'duration'])
        data['duration'] = data['duration'].astype(float)
        self.data = data.sort_values(by='duration')

    def __len__(self):
        return # YOUR CODE

    def __getitem__(self, idx):
        audio_path = self.data.audio_path[idx]
        text = self.data.text[idx]

        
        ### write your code here ###
        
        return {"audio":  audio,  # torch tensor, (num_timesteps)
                "audio_len": audio_len, # int
                "text": text, # str
                "text_len": text_len, # int
                'tokens': tokens, # torch tensor, (text_len)
               }

In [ ]:
! pytest tests/test_dataset.py

Давайте посмотрим, как будут выглядеть элементы полученного датасета.

In [ ]:
train_dataset = '/data/manifests/train.txt'
val_dataset = '/data/manifests/val.txt'
test_dataset = '/data/manifests/test.txt'

In [18]:
dataset = AudioDataset(train_dataset, vocab)

for i, element in enumerate(dataset):
    print(f'element number: {i}')
    pprint(element)
    print()
    if i > 1:
        break

element number: 0
{'audio': tensor([ 0.0000,  0.0000,  0.0000,  ...,  0.0005, -0.0008, -0.0006]),
 'audio_len': 8360,
 'text': 'один',
 'text_len': 4,
 'tokens': tensor([15,  4,  9, 14])}

element number: 1
{'audio': tensor([-1.3758e-09, -2.2855e-09, -1.3640e-09,  ..., -2.1114e-04,
        -4.1249e-04, -2.6402e-04]),
 'audio_len': 8640,
 'text': 'ноль',
 'text_len': 4,
 'tokens': tensor([14, 15, 12, 27])}

element number: 2
{'audio': tensor([0.0000, 0.0000, 0.0000,  ..., 0.0068, 0.0080, 0.0038]),
 'audio_len': 8987,
 'text': 'два',
 'text_len': 3,
 'tokens': tensor([4, 2, 0])}



Чтобы эффективно обучать нейронную сеть, необходимо подавать в нее данные батчами. В этом нам поможет функция `torch.utils.data.DataLoader`. 

Обратите внимание, что некоторые данные в датасете разной длины (например, `audio`), для формирования батча из таких данных  необходимо использовать паддинг. Для этого можно реализовать фукцию `collate_fn`. Подробнее про то, как именно использовать `collate_fn` можно почитатать в доках к `torch.utils.data.DataLoader`.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def collate_fn(batch):
    """ 
        Inputs:
            batch: list of elements with length=batch_ize
        Returns:
            dict
    """
    
    ### write your code here ###

    return {'audios': audios, # torch tensor, (batch_size, max_num_timesteps)
            'audio_lens': audio_lens, # torch tensor, (batch_size)
            'texts': texts,  # list, len=(batch_size)
            "text_lens": text_lens, # torch tensor, (batch_size)
            'tokens': tokens,  # torch tensor, (batch_size, max_text_len)
           }


In [ ]:
! pytest tests/test_collate_fn.py

In [ ]:
batch_size = 90
num_workers = 8

При создании DataLoader не забудьте использовать параметры `batch_size`, `num_workers`.

Так же в DataLoader есть параметр `shuffle`, который используется для перемешивания данных. Сейчас наши аудио отсортированы по длине, т.е. длины аудио внутри одного батча максимально близки друг другу - и это самый эффективный способ формировать батчи в распознавании речи. Если включить shuffle=True, то короткие аудио могут попасть в один батч с длинными, средний размер батча увеличится, это будет менее эффективно. Поэтому необходимо **перемешивать сформированные батчи**, а не элементы датасета.
    
Реализовать перемешивание батчей в PyTorch - это не самая простая задача, поэтому советую сейчас пропустить этот шаг.

In [ ]:
train_dataloader = # YOUR CODE
val_dataloader = # YOUR CODE

Давайте посмотрим, что у нас хранится в даталоадере. Обратите внимание, что из-за `batch_size=90` выведется достаточно много значений. Можно уменьшить `batch_size`, чтобы посмотреть на выход, но обучать сеть лучше с `batch_size=90`.

In [23]:
for i, element in enumerate(train_dataloader):
    print(f'element number: {i}')
    pprint(element)
    print()
    if i > 0:
        break

element number: 0
{'audio_lens': tensor([8360, 8640, 8987, 9196, 9196]),
 'audios': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.3758e-09, -2.2855e-09, -1.3640e-09,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  2.2861e-03,
          2.3552e-03,  9.7071e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.5030e-03,
         -1.5566e-03, -8.6835e-04]]),
 'text_lens': tensor([4, 4, 3, 5, 3]),
 'texts': ['один', 'ноль', 'два', 'шесть', 'нет'],
 'tokens': tensor([[15,  4,  9, 14,  0],
        [14, 15, 12, 27,  0],
        [ 4,  2,  0,  0,  0],
        [25,  5, 18, 19, 27],
        [14,  5, 19,  0,  0]])}

element number: 1
{'audio_lens': tensor([ 9405,  9405,  9600,  9823, 10032]),
 'audios': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0

# Акустические фичи

Как мы обсуждали на лекциях, есть разные способы построить аудио фичи. Мы будем использовать log mel spectrogram.


In [ ]:
def compute_log_mel_spectrogram(audio, sequence_lengths,
                            sample_rate=8000,
                            window_size=0.02,
                            window_step=0.01,
                            f_min=20,
                            f_max=3800,
                            n_mels=64,
                            window_fn=torch.hamming_window,
                            power=1.0,
                            eps=1e-6
                            ):
    """ Compute log-mel spectrogram.
        Input shape:
            audio: 3D tensor with shape (batch_size, num_timesteps)
            sequence_lengths: 1D tensor with shape (batch_size)
        Returns:
            4D tensor with shape (batch_size, n_mels, new_num_timesteps)
            1D tensor with shape (batch_size)
    """

    win_length = int(window_size * sample_rate)
    hop_length = int(window_step * sample_rate)
    
    ### write your code here ###
    
    return log_mel_spectrogram, sequence_lengths

In [ ]:
! pytest tests/test_compute_log_mel_spectrogram.py

Давайте посмотрим, как выглядит log-mel спектрограмма.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
audio, audio_len = open_audio('test_files/test_audio.opus', 8000)
spectrogram, new_len = compute_log_mel_spectrogram(audio, audio_len)
plt.pcolormesh(spectrogram)
plt.xlabel('T')
plt.ylabel('mels')
plt.show()
Audio(data=audio.numpy(), rate=sample_rate)

# Нейронная сеть

Мы подготовили все данные, теперь можно заняться реализацией нейронной сети. Будем реализовывать архитектуру [Deepspeech 2](https://arxiv.org/pdf/1512.02595.pdf) в немного упрощенном виде.


<img src="images/cat_reproduction.jpg" width="400" height="400">


Вот так будет выглядеть архитектура сети:


<img src="images/deepspeech.jpg" width="200" height="150">

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
    def __init__(self, num_mel_bins, hidden_size, num_layers, num_tokens):
        super(Model, self).__init__()

        conv1_params = AttrDict(
            {
                "num_filters": 32,
                "kernel_size": [21, 11],
                "stride": [1, 1] 
            })                             
        conv2_params = AttrDict(
            {
                "num_filters": 64,
                "kernel_size": [11, 11],
                "stride": [1, 3]
            }
        )
        
        self.conv1_params = conv1_params
        self.conv2_params = conv2_params

        self.conv = nn.Sequential(
            # CONV 1
            # BATCH NORM 1
            # RELU
            # CONV 2
            # BATCH NORM 2
            # RELU
            
            ### write your code here ###
        )

        
        rnn_input_size = # YOUR CODE

        # 4 слоя бидир lstm
        self.lstm = # YOUR CODE
        
        self.output_layer = # YOUR CODE

    def forward(self, inputs, seq_lens, state=None):
        """
            Input shape:
                audio: 3D tensor with shape (batch_size, num_mel_bins, num_timesteps)
                sequence_lengths: 1D tensor with shape (batch_size)
            Returns:
                3D tensor with shape (new_num_timesteps, batch_size, alphabet_len)
                1D tensor with shape (batch_size)
            """
        
        outputs = inputs.unsqueeze(1) # conv2d input should be four-dimensional
        
        ### write your code here ###


        seq_lens = self.get_new_seq_lens(seq_lens,
                                         self.conv1_params.kernel_size[1], self.conv1_params.stride[1],
                                         self.conv2_params.kernel_size[1], self.conv2_params.stride[1])
        return F.log_softmax(outputs, dim=-1), seq_lens

    @staticmethod
    def transpose_and_reshape(inputs):
    
        """ This function will be very useful for converting the output of a convolutional layer 
            to the input of a lstm layer
            
            Input shape:
                inputs: 4D tensor with shape (batch_size, num_filters, num_features, num_timesteps)
            Returns:
                3D tensor with shape (batch_size, num_timesteps, new_num_features)
            """
            
        sizes = inputs.size()
                
        # reshape # YOUR CODE
        # (batch_size, num_filters * num_features, num_timesteps)
        
        # transpose # YOUR CODE
        # (batch_size, num_timesteps, new_num_features)
        
        return outputs
           
    @staticmethod
    def get_new_seq_lens(seq_lens, conv1_kernel_size, conv1_stride, conv2_kernel_size, conv2_stride):
    
        """ Compute sequence_lengths after convolutions
            """
            
        ### write your code here ###
        return seq_lens

In [ ]:
num_mel_bins = 64
hidden_size= 512
num_layers = 4

In [ ]:
model = Model(num_mel_bins=num_mel_bins,
              hidden_size=hidden_size,
              num_layers=num_layers,
              num_tokens=num_tokens)

In [ ]:
! pytest tests/test_model.py

Инициализируем модель из чекпоинта, чтобы она обучилась быстрее.

In [ ]:
def load_from_ckpt(model, ckpt_path):
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
load_from_ckpt(model, '/data/ckpt.pt')

Тест, чтобы проверить, что код модели корректно написан, модель правильно восстановилась из чекпоинта

In [ ]:
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
output = model(*compute_log_mel_spectrogram(torch.unsqueeze(audio, 0), torch.unsqueeze(torch.tensor([audio_len]), 0)))
assert torch.isclose(output[0][0][0][0], torch.tensor(-3.53916406))
assert torch.isclose(output[0][15][0][30], torch.tensor(-3.605963468))

Отправим модель на гпу.

<img src="images/cuda_is_important.jpg" width="400" height="400">


In [ ]:
model.cuda(cuda_device)

# Обучаем модельку

In [ ]:
from time import time
from tqdm import tqdm_notebook as tqdm
import os

Давайте напишем несколько вспомогательных функций, которые будут нам нужны для обучения модели.

Для начала займемся метриками. Основная метрика - wer (word error rate).

Тут поможет библиотека `editdistance`.

In [ ]:
import editdistance

In [ ]:
def calc_wer(predicted_text, gt_text):
    """ Compute wer.
        Inputs:
            predicted_text: str
            gt_text: str
        Returns:
            wer: int
    """
    ### write your code here ###
    
    return wer


def calc_wer_for_batch(list_of_predicted_text, list_of_gt_text):
    """ Compute mean wer for batch.
            Inputs:
                list_of_predicted_text: list
                list_of_gt_text: list
            Returns:int
            
    """
    
    ### write your code here ###
    
    return mean_wer

In [ ]:
! pytest tests/test_compute_wer.py

Давайте реализуем greedy decoding. 

Сначала научимся получать greedy trn из выравниваний. Можно использовать `itertools`.

Не забудьте выкинуть лишние пробелы в начале и конце полученного текста!

In [ ]:
import itertools

In [ ]:
def decode(alignment):
    """ Get text from alignment.
        Inputs:
            alignment: str
        Returns:
            text: srt
    """
    ### write your code here ###
    
    return text

In [ ]:
! pytest tests/test_decode.py

Получим greedy text из выхода акустической модели (logprobs).

In [ ]:
def get_prediction(logprobs, logprobs_lens, vocab):
    """ Compute greedy text from loglikes.
            Input shape:
                logprobs: 3D tensor with shape (num_timesteps, batch_size, alphabet_len)
                logprobs_lens: 1D tensor with shape (batch_size)
            Returns:
                list of texts with len (batch_size)
        """
    ### write your code here ###
    
    return prediction

In [ ]:
! pytest tests/test_get_prediction.py

В этой функции надо из сырых данных, извлеченных из датасета, получить спектрограмму, прогнать через модель, посчитать средний лосс и wer для батча, list с текстами предсказанных гипотез для батча.

In [ ]:
def get_model_results(model, audio, audio_lens, tokens, text, text_lens, vocab, loss_fn):
    """ get mean loss, mean wer and prediction list for batch
        Returns:
            loss: int
            wer: int
            prediction: list of str
            
    """
    
    ### write your code here ###
    
    return loss, wer, prediction

Для удобства будем логировать метрики в [tensorboard](https://pytorch.org/docs/stable/tensorboard.html).

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class TensorboardLogger:
    def __init__(self, tensorboard_path):
        self.writer = SummaryWriter(tensorboard_path)

    def log(self, step, loss, wer, mode):
        assert mode in ('train', 'val')
        
        ### write your code here ###
        
        # add loss to tb 
        # add wer to tb 

    def log_text(self, step, pred_texts, gt_texts, mode):
        
        ### write your code here ###
        
        for pred_text in pred_texts:
            # add pred text to tb 
            
        for gt_text in gt_texts:
            # add gt text to tb 
      

    def close(self):
        self.writer.close()

Теперь можем собрать функции в train loop. 

In [ ]:
def training(model, optimizer, loss_fn, num_epochs, train_dataloader, val_dataloader, log_every_n_batch, model_dir,
             vocab):

    logger = TensorboardLogger(model_dir)

    for epoch in range(num_epochs):

        start_time = time()
        train_loss, train_wer = 0, 0
        model.train(True)

        for iteration, batch in enumerate(tqdm(train_dataloader)):
            loss, wer, prediction = get_model_results(model, batch["audios"], batch["audio_lens"],
                                                      batch["tokens"], batch["texts"],
                                                      batch["text_lens"], vocab, loss_fn)



            # optimizer step
            ### write your code here ###
             
             
            

            train_loss += loss.cpu().data.numpy()
            train_wer += wer

            step = len(train_dataloader) * epoch + iteration
            if step % log_every_n_batch == 0:
                logger.log(step, loss, wer, 'train')
                logger.log_text(step, prediction, batch["texts"], "train")

        train_loss /= len(train_dataloader)
        train_wer /= len(train_dataloader)

        val_loss, val_wer = 0, 0
        model.train(False)

        for batch in tqdm(val_dataloader):
            loss_val, wer, prediction = get_model_results(model, batch["audios"], batch["audio_lens"],
                                                          batch["tokens"], batch["texts"],
                                                          batch["text_lens"], vocab, loss_fn)
            val_loss += loss_val.cpu().data.numpy()
            val_wer += wer

        val_loss /= len(val_dataloader)
        val_wer /= len(val_dataloader)

        logger.log(step, val_loss, val_wer, 'val')
        logger.log_text(step, prediction, batch["texts"], "val")

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, os.path.join(model_dir, f'epoch_{epoch}.pt'))

        print(f'\nEpoch {epoch + 1} of {num_epochs} took {time() - start_time}s, ' + \
              f'train loss: {train_loss}, val loss: {val_loss}, train wer: {train_wer}, val wer: {val_wer}')

    logger.close()
    print("Finished!")

In [ ]:
num_epochs = 7
model_dir = 'models/1'
log_every_n_batch = 10

In [ ]:
learning_rate = 2e-4
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
loss_fn = torch.nn.CTCLoss(blank=blank_index, reduction='mean')

Про ctc loss очень хорошо написано [тут](https://distill.pub/2017/ctc/). А [это](https://www.cs.toronto.edu/~graves/icml_2006.pdf) исходная статья.

Если не используете перемешивание батчей (шафл), то при подборе batch size обратите внимание, что данные отсортированы (обучение будет замедляться с увеличением длины аудио).

In [ ]:
training(model, opt, loss_fn, num_epochs, train_dataloader, val_dataloader, log_every_n_batch,
             model_dir, vocab)

<img src="images/training.jpeg" width="400" height="400">


После того, как напишете весь код и запустите обучение вашей первой модели, примерно после 5-6 эпох качество модели достигнет 30-35% WER.

# Как можно улучшить полученные результаты

<img src="images/pronunciation_or_not.jpg" width="400" height="400">

Далее описаны несколько способов, которые могут помочь улучшить качество. Потенциальный прирост обозначим ★, чем больше звездочек, тем более хорошее улучшение качества можно ожидать.

## Beam search ★★

На лекции обсуждали, что beam search помогает достичь более хорошего качества, чем greedy декодирование.

Изучить алгоритм можно [тут](https://medium.com/corti-ai/ctc-networks-and-language-models-prefix-beam-search-explained-c11d1ee23306) и [тут](https://drive.google.com/viewerng/viewer?url=https://arxiv.org/pdf/1408.2873.pdf)

Код для beam search посмотреть [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/decoders_deprecated.py). (Отредактировать при необходимости )

## Внешняя языковая модель ★★★


Внешняя языковая модель позволяет улучшить качество, т.к. убирает условную независимость соседних символов, которая свойственна ctc лоссу.

Этот пункт состоит из двух этапов: сначала надо обучить языковую модель, затем встроить ее в beam search.

#### Обучение

Для обучения n-gram языковой модели можно использовать фреймворк kenlm.
Документация: 
* https://kheafield.com/code/kenlm/
* https://github.com/kpu/kenlm

Модель сначала строится в формате arpa, затем ее лучше перевести в формат trie. Вызывать полученную модель можно через питон [ссылка](https://github.com/kpu/kenlm#python-module)


#### Данные

Для обучения модели конечно нужны данные :) Тут есть варианты:

1. Можно обучить маленькую языковую модель на текстах из акустических обучающих данных (из трейна!).

минусы: этих данных мало

плюсы: домен остается таким же

2. Можно взять внешние данные, например, [отсюда](http://data.statmt.org/cc-100/). (46G).

минусы: тексты из другого (произвольного) домена

плюсы: данных много

При необходимости, данные надо предобработать - привести к нижнему регистру, разделить на предложения. Убрать предложения, которые содержат символы не из русского алфавита 


#### Внедрение в beam search

В разделе про beam search есть ссылки на алгоритм.

Можно использовать аргумент `ext_scoring_func` [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/decoders_deprecated.py#L47).

Пример скорера можно найти [тут](https://github.com/PaddlePaddle/DeepSpeech/blob/master/decoders/scorer_deprecated.py)

## Аугментации ★


Позволяют искусственно увеличь размер обучающей выборки, сделать его более разнообразным.

### Аугментации аудио 

Применяются к аудиосигналу. Аугментации обычно реализуются через [sox](https://ru.wikipedia.org/wiki/SoX_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0)). [Тут](http://sox.sourceforge.net/sox.html#EFFECTS) можно посмотротреть полный список sox эффектов с описанием. 

Полный список sox эффектов, доступных в torchaudio, можно посмотреть [тут](https://github.com/pytorch/audio/issues/260).

Эффекты можно комбинировать.

Внимание!

* Аугментации надо применять очень аккуратно (!) - слишком сильные аугментации только ухудшат качество. Лучше применять аугментации с некоторой вероятностью.
* Применять **только на обучающую выборку, не на валидацию!**
* Некоторые аугментации меняют sample rate и длину аудио.
* Можно применять не открывая предварительно аудиофайл [ссылка](https://pytorch.org/audio/stable/sox_effects.html#applying-effects-on-file).


Примеры:

In [ ]:
def augment_audio(audio, sample_rate, effects):
    
    effects = [effects, ['rate', '8000']]

    augmented_audio, sample_rate = torchaudio.sox_effects.apply_effects_tensor(
        torch.unsqueeze(audio, 0),
        sample_rate=sample_rate,
        effects=effects,
        channels_first=True)
    
    return augmented_audio

In [ ]:
sample_rate = 8000
audio, audio_len = open_audio('test_files/test_audio.mp3', sample_rate)
Audio(data=audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['treble', '20'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['bass', '20'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['pitch', '400'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['speed', '1.5'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

In [ ]:
augmented_audio = augment_audio(audio, sample_rate, ['tempo', '1.5'])
Audio(data=augmented_audio.numpy(), rate=sample_rate)

### Аугментации спектрограммы

[SpecAugment: A Simple Data Augmentation Method for Automatic Speech Recognition](https://arxiv.org/pdf/1904.08779.pdf)

Методов [Frequency masking](https://pytorch.org/audio/stable/transforms.html#frequencymasking) и [Time masking](https://pytorch.org/audio/stable/transforms.html#timemasking) должно быть достаточно.

# Что еще можно попробовать: 

1. Поэкспериментировать с learning rate, оптимайзером (например, взять SGD). Можно добавить [lr decay](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ExponentialLR) ★
2. Поэкспериментировать с нейронной сетью и восттановлением из чекпоинта. Например, зафиксировать предобученные слои и дообучить остальные, потом с маленьким learning rate дообучить всю модель. ★
3. Добавить новые слои в нейронную сеть. ★★
4. Использовать больше данных. ★★★
<img src="images/more_data.jpg" width="400" height="400">

В этом случае для ускорения можно запустить распределенное обучение на нескольких gpu с помощью [horovod](https://github.com/horovod/horovod). Данные можно взять тут:

* [open_stt](https://github.com/snakers4/open_stt) (до 2.5 TB данных!)
* [Russian LibriSpeech](https://openslr.org/96/) (9 GB данных)


5\. Не использовать чекпоинт и обучить свою сеть :) ★ (потребуется больше данных!)

6\. Shallow fusion - можно обучить дополнительную языковую модель и использовать в качестве рескорера 
[ссылка](https://arxiv.org/pdf/1503.03535.pdf). ★

7\. Реализовать перемешивание батчей. ★
